## DigiSafe
---

In [2]:
:cache 1000
:dep apache-avro
:dep base64
:dep chacha20poly1305
:dep lz4
:dep reed-solomon
:dep rust-argon2
:dep serde = { features = ["derive"] }
:dep sha3
:dep getrandom

cache: 1000 MiB


In [3]:
pub mod Database {

    // Database<Unlocked> <-> Avro <-> LZ4 <-> ChaCha20Poly1305 <-> Base64 <-> Database<Locked>
    // Database<Locked> -> Avro -> Reed-Solomon -> Vec<u8> -> name.digisafe

    use serde::{Deserialize, Serialize};
    use std::collections::HashMap;
    use std::io::{Read, Write};

    pub fn hash_lite(msg: &Vec<u8>) -> Vec<u8> {
        use sha3::{Digest, Sha3_256};
        Sha3_256::digest(Sha3_256::digest(msg)).to_vec()
    }

    pub fn hash_heavy(msg: String) -> Vec<u8> {
        use argon2::{Config, Variant, Version, hash_raw, verify_raw};
        let lite_hash = hash_lite(&msg.into());
        let salt = b"digisafe";
        let config = Config {
            variant: Variant::Argon2id,
            version: Version::Version13,
            mem_cost: 1048576,
            time_cost: 2,
            lanes: 4,
            secret: &[],
            ad: &[],
            hash_length: 256
        };
        let heavy_hash = argon2::hash_raw(&lite_hash[..], salt, &config).unwrap();
        hash_lite(&heavy_hash)
    }

    pub fn to_base64(msg: &Vec<u8>) -> String {
        use base64::{Engine as _, engine::general_purpose};
        general_purpose::STANDARD_NO_PAD.encode(msg)
    }

    pub fn from_base64(msg_enc: &String) -> Vec<u8> {
        use base64::{Engine as _, engine::general_purpose};
        general_purpose::STANDARD_NO_PAD.decode(msg_enc).unwrap()
    }

    pub fn compress(msg: Vec<u8>) -> Vec<u8> {
        use lz4::{Decoder, EncoderBuilder};
        let mut encoder = EncoderBuilder::new().level(9).build(vec![]).unwrap();
        encoder.write(&msg[..]);
        encoder.finish().0
    }

    pub fn decompress(msg_enc: Vec<u8>) -> Vec<u8> {
        use lz4::Decoder;
        let mut msg = vec![];
        {
            let mut decoder = Decoder::new(&msg_enc[..]).unwrap();
            std::io::copy(&mut decoder, &mut msg);
        }
        msg
    }
    
    pub fn encrypt(msg: Vec<u8>, key: Vec<u8>, nonce: Vec<u8>) -> Vec<u8> {
        use chacha20poly1305::{aead::{Aead, KeyInit}, XChaCha20Poly1305};
        let key: [u8; 32] = key.try_into().unwrap();
        let nonce: [u8; 24] = nonce.try_into().unwrap();
        let cipher = XChaCha20Poly1305::new(&key.into());
        cipher.encrypt(&nonce.into(), &msg[..]).unwrap()
    }

    pub fn decrypt(msg_enc: Vec<u8>, key: Vec<u8>, nonce: Vec<u8>) -> Option<Vec<u8>> {
        use chacha20poly1305::{aead::{Aead, KeyInit}, XChaCha20Poly1305};
        let key: [u8; 32] = key.try_into().unwrap();
        let nonce: [u8; 24] = nonce.try_into().unwrap();
        let cipher = XChaCha20Poly1305::new(&key.into());
        if let Ok(msg) = cipher.decrypt(&nonce.into(), msg_enc.as_ref()) {
            Some(msg)
        } else {
            None
        }
    }

    const unlocked_schema_raw: &str = r#"
    {
        "type": "record",
        "name": "unlocked",
        "fields": [
            {"name": "db", "type": {"type": "map", "values": "string"}},
            {"name": "info", "type": {"type": "map", "values": "string"}}
        ]
    }
    "#;

    #[derive(Debug, Deserialize, Serialize)]
    pub struct AvroUnlocked {
        db: HashMap<String, String>,
        info: HashMap<String, String>,
    }
    
    const locked_schema_raw: &str = r#"
    {
        "type": "record",
        "name": "locked",
        "fields": [
            {"name": "db", "type": "string"},
            {"name": "info", "type": {"type": "map", "values": "string"}}
        ]
    }
    "#;

    #[derive(Debug, Deserialize, Serialize)]
    pub struct AvroLocked {
        db: String,
        info: HashMap<String, String>,
    }

    #[derive(Default, Debug)]
    pub struct Unlocked {
        db: HashMap<String, String>,
        info: HashMap<String, String>,
        key: Vec<u8>,
    }

    #[derive(Debug, Default)]
    pub struct Locked {
        db: String,
        info: HashMap<String, String>,
    }


    impl AvroUnlocked {
        pub fn to_unlocked_struct(self, key: Vec<u8>) -> Unlocked {
             Unlocked {
                 db: self.db,
                 info: self.info,
                 key: key,
             }
        }
    }

    impl AvroLocked {
        pub fn to_locked_struct(self) -> Locked {
             Locked {
                 db: self.db,
                 info: self.info,
             }
        }
    }
    
    impl Unlocked {
        
        pub fn new(key: impl Into<String>) -> Self {
            let mut obj = Unlocked {
                db: Default::default(),
                info: Default::default(),
                key: hash_heavy(key.into()),
            };
            let mut randy = [0u8; 32];
            getrandom::getrandom(&mut randy);
            obj.set_info("uid", to_base64(&randy.to_vec()));
            obj
        }
        
        pub fn lock(self) -> Locked {
            let cadb = compress(self.to_avro());
            let nonce = hash_lite(&cadb)[..24].to_vec();
            let mut info = HashMap::<String, String>::new();
            info.extend(self.info);
            info.insert("nonce".into(), to_base64(&nonce));
            let db = to_base64(&encrypt(cadb, self.key, nonce));
            Locked {
                db: db,
                info: info,
            }
        }
    
        pub fn get(&self, akey: &String) -> Option<String> {
            self.db.get(akey).cloned()
        }
    
        
        pub fn set(&mut self, akey: impl Into<String>, aval: impl Into<String>) {
            self.db.insert(akey.into(), aval.into());
        }
    
        pub fn get_info(&self, akey: &String) -> Option<String> {
            self.info.get(akey).cloned()
        }
    
        
        pub fn set_info(&mut self, akey: impl Into<String>, aval: impl Into<String>) {
            self.info.insert(akey.into(), aval.into());
        }

        pub fn merge(&mut self, other: Unlocked) {
            self.db.extend(other.db);
        }

        pub fn to_avro_struct(&self) -> AvroUnlocked {
            AvroUnlocked {
                db: self.db.clone(),
                info: self.info.clone(),
            }
        }

        pub fn to_avro(&self) -> Vec<u8> {
            use apache_avro::{to_value, Schema, Writer};
            let schema = Schema::parse_str(unlocked_schema_raw).unwrap();
            let mut writer = Writer::new(&schema, Vec::new());
            writer.append(to_value(&self.to_avro_struct()).unwrap()).unwrap();
            let avro_dat = writer.into_inner().unwrap();
            avro_dat
        }

        pub fn from_avro(avro_dat: Vec<u8>, key: Vec<u8>) -> Unlocked {
            use apache_avro::{from_value, Reader};
            let reader = Reader::new(&avro_dat[..]).unwrap();
            let db = from_value::<AvroUnlocked>(&reader.last().unwrap().unwrap()).unwrap();
            db.to_unlocked_struct(key)
        }
    }

    /*
    impl std::fmt::Debug for Unlocked {
        fn fmt(&self, objf: &mut std::fmt::Formatter) -> std::fmt::Result {
            write!(objf, "Database::Unlocked: {{ redacted }}")
        }
    }
    */

    impl Locked {
        
        pub fn unlock(self, password: impl Into<String>) -> Option<Unlocked> {
            let key = hash_heavy(password.into());
            let nonce = from_base64(&self.info.get("nonce").unwrap());
            let cradb = decrypt(from_base64(&self.db), key.clone(), nonce);
            if cradb.is_none() {
                return None;
            }
            let radb = decompress(cradb.unwrap());
            let db = Unlocked::from_avro(radb, key);
            Some(db)
        }

        pub fn to_avro_struct(&self) -> AvroLocked {
            AvroLocked {
                db: self.db.clone(),
                info: self.info.clone(),
            }
        }

        pub fn to_avro(&self) -> Vec<u8> {
            use apache_avro::{to_value, Schema, Writer};
            let schema = Schema::parse_str(locked_schema_raw).unwrap();
            let mut writer = Writer::new(&schema, Vec::new());
            writer.append(to_value(&self.to_avro_struct()).unwrap()).unwrap();
            let avro_dat = writer.into_inner().unwrap();
            avro_dat
        }

        pub fn from_avro(avro_dat: Vec<u8>) -> Locked {
            use apache_avro::{from_value, Reader};
            let reader = Reader::new(&avro_dat[..]).unwrap();
            let db = from_value::<AvroLocked>(&reader.last().unwrap().unwrap()).unwrap();
            db.to_locked_struct()
        }


    }

}

In [4]:
let mut db = Database::Unlocked::new("test_key");
db

Unlocked { db: {}, info: {"uid": "ie0g6a8g8Gff4qxK6TXhffz8XmSgDKJaY2xlT57lleE"}, key: [31, 237, 125, 123, 74, 172, 79, 110, 57, 22, 0, 231, 83, 11, 63, 96, 30, 79, 197, 117, 173, 128, 49, 12, 161, 121, 237, 163, 211, 88, 187, 138] }

In [5]:
db.set_info("name", "test_db");
db

Unlocked { db: {}, info: {"name": "test_db", "uid": "ie0g6a8g8Gff4qxK6TXhffz8XmSgDKJaY2xlT57lleE"}, key: [31, 237, 125, 123, 74, 172, 79, 110, 57, 22, 0, 231, 83, 11, 63, 96, 30, 79, 197, 117, 173, 128, 49, 12, 161, 121, 237, 163, 211, 88, 187, 138] }

In [6]:
db.set("k1", "v1");
db.set("k2", "v2");
db.set("k3", "v3");
db

Unlocked { db: {"k3": "v3", "k1": "v1", "k2": "v2"}, info: {"name": "test_db", "uid": "ie0g6a8g8Gff4qxK6TXhffz8XmSgDKJaY2xlT57lleE"}, key: [31, 237, 125, 123, 74, 172, 79, 110, 57, 22, 0, 231, 83, 11, 63, 96, 30, 79, 197, 117, 173, 128, 49, 12, 161, 121, 237, 163, 211, 88, 187, 138] }

In [7]:
let db = db.lock();
db

Locked { db: "zph6W30jXXEffe+IM0FLPsAPxk+p7INLPW15V46mHwQJTl7kPCEctvUpqB0zRQMrboHJQMWJZjC8oKppD0pSCJmxx6yEePk5ODwrQovSDwM+NsDWDQL58QjO5GIqVgZIrELZ+XDtBCjoRVQC6LgJ0UnK4fY7ROxHZ6pNyIgM/JW/oPvw8y9bAvfD6ajIu+Lo1PyeFyXFZNgjBCALOFpLPB4T6Ioumzk8Udfp3BynbT973JkW7UAEu+1fj5i8aH5VEq29xsqQaEoKrK/+d+do4rCz4pugQxaz+hdw5Y1NdbMBO/H8wrlK2NYozeGavB8QcDO14B2EBFxnKg1/dn2MCKCLX4uc4Iff53JfHCaeBHYkWfjX", info: {"name": "test_db", "uid": "ie0g6a8g8Gff4qxK6TXhffz8XmSgDKJaY2xlT57lleE", "nonce": "vGT3pmOq1d2T9kjg7wm/+I970U4S4/nJ"} }

In [8]:
let db = db.unlock("test_key").unwrap();
db

Unlocked { db: {"k3": "v3", "k1": "v1", "k2": "v2"}, info: {"uid": "ie0g6a8g8Gff4qxK6TXhffz8XmSgDKJaY2xlT57lleE", "name": "test_db"}, key: [31, 237, 125, 123, 74, 172, 79, 110, 57, 22, 0, 231, 83, 11, 63, 96, 30, 79, 197, 117, 173, 128, 49, 12, 161, 121, 237, 163, 211, 88, 187, 138] }

In [9]:
let db = db.lock();
db

Locked { db: "l+ZnzFynzGfdY5kSAxmz1Z537V6EY6GY9pzPJG8KoMx7hR60FWO6kjDY4FMaiB4OyiOt4C96+OPK0PooXWxFBDMPS2dJ2nBkm+VFKlrmJBQdIbL2IcZTqqF43KyiDgH0wHYLM3jsSFfudROMllejV3rPANFbH/YZx5gp7Sw49oid3qyrB7FstUcTBv3SDvBsyXJcj5Xov0PWeNvk+QzIT3ST4NCLYfDn6EN1gb5tDF06vcNX+zsYIile857s85vDaVH3bsHXzZNfyQrUnUyXkTN0uJ1w64PJvqZIkziIpQgn8bZGnQo0WAl5asfy6puON9RAtbByWZJl9Ujf9LJOISA4LLG7cNdDhHd8KoHD9WBgpquGzA", info: {"nonce": "Nz8maHPgyKTn+yaqkg9zMvIcaiuOy1uq", "uid": "ie0g6a8g8Gff4qxK6TXhffz8XmSgDKJaY2xlT57lleE", "name": "test_db"} }

In [10]:
db.to_avro()[..100]

[79, 98, 106, 1, 4, 22, 97, 118, 114, 111, 46, 115, 99, 104, 101, 109, 97, 132, 2, 123, 34, 116, 121, 112, 101, 34, 58, 34, 114, 101, 99, 111, 114, 100, 34, 44, 34, 110, 97, 109, 101, 34, 58, 34, 108, 111, 99, 107, 101, 100, 34, 44, 34, 102, 105, 101, 108, 100, 115, 34, 58, 91, 123, 34, 110, 97, 109, 101, 34, 58, 34, 100, 98, 34, 44, 34, 116, 121, 112, 101, 34, 58, 34, 115, 116, 114, 105, 110, 103, 34, 125, 44, 123, 34, 110, 97, 109, 101, 34, 58]

In [11]:
let db = Database::Locked::from_avro(db.to_avro());
db

Locked { db: "l+ZnzFynzGfdY5kSAxmz1Z537V6EY6GY9pzPJG8KoMx7hR60FWO6kjDY4FMaiB4OyiOt4C96+OPK0PooXWxFBDMPS2dJ2nBkm+VFKlrmJBQdIbL2IcZTqqF43KyiDgH0wHYLM3jsSFfudROMllejV3rPANFbH/YZx5gp7Sw49oid3qyrB7FstUcTBv3SDvBsyXJcj5Xov0PWeNvk+QzIT3ST4NCLYfDn6EN1gb5tDF06vcNX+zsYIile857s85vDaVH3bsHXzZNfyQrUnUyXkTN0uJ1w64PJvqZIkziIpQgn8bZGnQo0WAl5asfy6puON9RAtbByWZJl9Ujf9LJOISA4LLG7cNdDhHd8KoHD9WBgpquGzA", info: {"nonce": "Nz8maHPgyKTn+yaqkg9zMvIcaiuOy1uq", "uid": "ie0g6a8g8Gff4qxK6TXhffz8XmSgDKJaY2xlT57lleE", "name": "test_db"} }

In [12]:
let db = db.unlock("test_key").unwrap();
db

Unlocked { db: {"k3": "v3", "k1": "v1", "k2": "v2"}, info: {"name": "test_db", "uid": "ie0g6a8g8Gff4qxK6TXhffz8XmSgDKJaY2xlT57lleE"}, key: [31, 237, 125, 123, 74, 172, 79, 110, 57, 22, 0, 231, 83, 11, 63, 96, 30, 79, 197, 117, 173, 128, 49, 12, 161, 121, 237, 163, 211, 88, 187, 138] }

In [13]:
// todo: Locked -> AvroLocked -> Reed-Solomon -> Vec<u8> -> name.digisafe